In [1]:
from io import BytesIO
import json
import PIL.Image
import numpy as np
import requests

from velour.client import Client
from velour.data_types import GroundTruthInstanceSegmentation, GroundTruthSemanticSegmentation, Image, Label
from velour.viz import combined_segmentation_mask
from velour.integrations.coco import upload_coco_panoptic
from velour.metrics import Task

/home/czaloom/velour/.env-velour/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from chariot.datasets import Dataset

In [3]:
client = Client("http://localhost:8000")

Succesfully connected to http://localhost:8000/.


# Dataset ingestion

We assume the COCO panoptic evaluations from
http://images.cocodataset.org/annotations/panoptic_annotations_trainval2017.zip have been downloaded and unzipped (including unzipping the file `panoptic_val2017.zip`) to `./coco`

In [ ]:
# creates a dataset object in velour
dataset = client.create_image_dataset("coco2017-panoptic")

In [ ]:
with open("./coco/annotations/panoptic_val2017.json") as f:
    annotations = json.load(f)
    
annotations["annotations"] = annotations["annotations"][:10]

In [ ]:
# helper method to download underlying images from COCO
image_id_to_coco_url = {str(img_dict["id"]): img_dict["coco_url"] for img_dict in annotations["images"]}
def download_image(img: Image) -> PIL.Image:
    url = image_id_to_coco_url[img.uid]
    img_data = BytesIO(requests.get(url).content)
    return PIL.Image.open(img_data)

In [ ]:
upload_coco_panoptic(
    dataset,
    annotations=annotations,
    masks_path="./coco/annotations/panoptic_val2017/"
)

In [ ]:
dataset.get_images()

In [ ]:
download_image(dataset.get_images()[0])

In [ ]:
instance_segs = dataset.get_groundtruth_instance_segmentations("139")
semantic_segs = dataset.get_groundtruth_semantic_segmentations("139")

In [ ]:
instance_mask, instance_legend = combined_segmentation_mask(instance_segs, label_key="name")

In [ ]:
instance_mask

In [ ]:
for k, v in instance_legend.items():
    print(k)
    display(v)

In [ ]:
semantic_mask, semantic_legend = combined_segmentation_mask(semantic_segs, label_key="name")

In [ ]:
semantic_mask

In [ ]:
for k, v in semantic_legend.items():
    print(k)
    display(v)

# Model inference

In [4]:
from chariot.client import connect
from chariot.models import Model
from tqdm.auto import tqdm

from velour.integrations.chariot import create_model_from_chariot, parse_chariot_object_detections

In [5]:
connect("https://production.chariot.striveworks.us/")

In [ ]:
dataset = client.get_dataset("coco2017-panoptic")

In [13]:
model_name = "fasterrcnnresnet-50fpn"
chariot_model = Model(name=model_name, project_name="Global")

# velour_model = create_model_from_chariot(client, chariot_model)
velour_model = client.get_model('chariot-fasterrcnnresnet-50fpn-v1')

ClientException: Model with name 'chariot-fasterrcnnresnet-50fpn-v' does not exist.

In [12]:
for image_metadata in tqdm(dataset.get_images()):
    image = download_image(image_metadata)
    detections = chariot_model.detect(image)
    velour_dets = parse_chariot_object_detections(detections, image_metadata, label_key="name")
    velour_model.add_predictions(dataset=dataset, predictions=velour_dets)

  0%|          | 0/1 [00:00<?, ?it/s]


NameError: name 'dataset' is not defined

# Evaluation

for provenance, the dataset and inferences on the dataset must be finalized before evaluation can be done

In [ ]:
dataset.finalize()

In [ ]:
velour_model.finalize_inferences(dataset)

In [ ]:
eval_job = velour_model.evaluate_ap(
    dataset=dataset,
    model_pred_task_type=Task.BBOX_OBJECT_DETECTION,
    dataset_gt_task_type=Task.INSTANCE_SEGMENTATION,
)

In [ ]:
eval_job.status()

In [ ]:
eval_job.status()

In [ ]:
eval_job = velour_model.evaluate_ap(
    dataset=dataset,
    model_pred_task_type=Task.BBOX_OBJECT_DETECTION,
    dataset_gt_task_type=Task.INSTANCE_SEGMENTATION,
    max_area=300*30
)

In [ ]:
eval_job.status()

In [ ]:
eval_job.metrics()[:10]